BiLSTMVertical:
This is a module that implements a Bidirectional Long Short-Term Memory (BiLSTM) layer with a vertical direction of computation.
It takes two arguments: input_size and hidden_size, which determine the input and hidden state dimensions of the LSTM.
Inside the __init__ method, it initializes an nn.LSTM module with the specified input and hidden sizes. It's set to be bidirectional (bidirectional=True) and accepts input data in batches (batch_first=True).
In the forward method, it applies the BiLSTM to the input tensor x and returns the output.
BiLSTMHorizontal:
This is another module similar to BiLSTMVertical, but it implements a BiLSTM layer with a horizontal direction of computation.
It has the same constructor and forward method structure as BiLSTMVertical.
FullyConnectedLayer:
This module represents a fully connected (linear) layer in a neural network.
It takes two arguments: input_size and output_size, determining the input and output dimensions of the linear layer.
In the __init__ method, it initializes an nn.Linear module with the specified input and output sizes.
In the forward method, it applies the linear transformation to the input tensor x and returns the output.

In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
class BiLSTMVertical(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(BiLSTMVertical, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, bidirectional=True, batch_first=True)
        
        
    def forward(self, x):
        output, _ = self.lstm(x)
        return output
    

class BiLSTMHorizontal(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(BiLSTMHorizontal, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, bidirectional=True, batch_first=True)
       
        
    def forward(self, x):
       
        output, _ = self.lstm(x)
        return output
    
    
class FullyConnectedLayer(nn.Module):
    def __init__(self, input_size, output_size):
        super(FullyConnectedLayer, self).__init__()
        self.fc = nn.Linear(input_size, output_size)
       
        # Apply He initialization to linear layer weights
        init.kaiming_normal_(self.fc.weight, mode='fan_out', nonlinearity='relu')


        
    def forward(self, x):
        output = self.fc(x)
        return output



Sequencer2D
Is a sequencer in which the input data (images) are splitted into vertical adn horiziontal component, then vertical data will be passed as input to BilstmVertical ,while horizontal_data will be passed to BiLSTMHorizontal, to do so the input data are first processed in such a way we'll have a tensor with vertical data of each image and another tensor with horizontal data of each image. Oncee we get the outputs from both BiLSTMvertical and BiLSTMHorizontal, we 'll concatenate them and pass as input the result to Fully Connected Layer . At the end of the sequencer the results will be a list of tensor ( each tensor has inside data belongs to each image processed above)

In [2]:


class Sequencer2D(nn.Module):
    def __init__(self, input_size, hidden_size, mlp_input_size, mlp_output_size):
        super(Sequencer2D, self).__init__()

       
        vertical_bilstm = BiLSTMVertical(input_size, hidden_size)
        horizontal_bilstm = BiLSTMHorizontal(input_size, hidden_size)
        fully_connected = FullyConnectedLayer( mlp_input_size,mlp_output_size)  

        # Move models to GPU if available
        #vertical_bilstm.to(device)
        #horizontal_bilstm.to(device)
        #fully_connected.to(device)

        self.vertical_bilstm = vertical_bilstm
        self.horizontal_bilstm = horizontal_bilstm
        self.fully_connected = fully_connected
        
       # Batch normalization layers
        self.bn_vertical = nn.BatchNorm1d(hidden_size * 2)
        self.bn_horizontal = nn.BatchNorm1d(hidden_size * 2)
        self.bn_fc = nn.BatchNorm1d(mlp_output_size)


    def forward(self, x):
        if isinstance(x, list):
            x = torch.stack(x)
        if len(x.shape)==4:
            batch_size,channel, height, width = x.size()
            x=x.squeeze(1)
        elif len(x.shape) == 3 :
            batch_size, height, width = x.size()
        output=[]
        for index in range(batch_size):
            
            x_vertical = x[index][:, :height//2]  #split the image vertically
            x_horizontal = x[index][ :,width//2:]   #split image horizontally

            vertical_data = x_vertical.permute( 1,0).clone() #permutation to fit bilstm
            horizontal_data = x_horizontal.permute( 1,0).clone()#same here
            
            
            
            H_ver = self.vertical_bilstm(vertical_data)
            H_hor = self.horizontal_bilstm(horizontal_data)
            
            
            H_ver = H_ver.permute(1,0).clone() #permute to fit torch.cat
            H_hor = H_hor.permute(1,0).clone()
            
           
            
            
            H_concat = torch.cat((H_ver, H_hor), dim=1)#concatenaiton of the horizontal and vertical bilstm outputs
            H_concat = H_concat.unsqueeze(0) #set dimension to fit fully conncted layer

            
            
            
            
            
            
            output_pieces = self.fully_connected(H_concat)
            
            #output_pieces=torch.tensor(output_pieces, dtype=torch.float32)
            output.append(output_pieces) #add output to lists of each image data processed here
        
        return output


Patch Embedding : this costum function given an image , a patch size, will splitted the image in image_size/patch_size pieces.

image_size: The size of the input image. It represents both the height and width of the image.
patch_size: The size of each non-overlapping patch that the image is divided into.
in_channels: The number of input channels in the image. For RGB images, this is typically 3 (for the red, green, and blue channels).
embed_dim: The dimension of the embedded representations for each patch.


In [3]:
import torch.nn.init as init

class PatchEmbedding(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, embed_dim):
        super(PatchEmbedding, self).__init__()
        self.image_size = image_size
        self.patch_size = patch_size
        self.in_channels = in_channels
        self.embed_dim = embed_dim

        
        self.num_patches = (image_size // patch_size) ** 2 #compute the number of patches in such a way they won't overlap

        self.projection = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size) #convolutional layer takes the input image, divides it into patches, and embeds each patch into a lower-dimensional representation.
        self.bn = nn.BatchNorm2d(embed_dim)
        
    def forward(self, x):
        x = self.projection(x)
        B, C, H, W = x.size()
        x = x.reshape(B, self.num_patches, C, -1).permute(0, 2, 1, 3).clone()
        x = x.flatten(2).transpose(1, 2)
        return x

    def output_dimension(self):
        return self.embed_dim * self.num_patches


PW Linear Layer compute  pointwise linear transformations on a list of input tensors.

In [4]:
class PWLinearLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super(PWLinearLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.linear = nn.Linear(in_features, out_features)
        self.bn = nn.BatchNorm1d(out_features)
        
    def forward(self, input_list):
        # output_list = []
        # for input_tensor in input_list:
        #     output_tensor = self.linear(input_tensor)
        #     output_list.append(output_tensor)
        # stacked_output = torch.stack(output_list, dim=0)
        stacked_output = torch.cat(input_list, dim=0)
        # Apply linear transformation
        output_tensor = self.linear(stacked_output)
        return output_tensor


Patch merging, take the images splitted in patches (for examples by patch embedding ) and merge it together, the number of patches merged is based on the output channlee and  the  scale factor number.

In [5]:
class PatchMerging(nn.Module):
    def __init__(self, in_channels, out_channels, scale_factor):
        super(PatchMerging, self).__init__()
        self.scale_factor = scale_factor
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.bn = nn.BatchNorm1d(out_channels)
       

    def forward(self, patch_list):
        
        x = torch.cat(patch_list, dim=1).permute(1,2,0).clone() # compination of the list of patches into a single tensor + peermutation to fit convolutional layer
        
        
        x = self.conv(x) # Apply to the input x, a 1x1 convolution to merge the patches together
        
        
        x = nn.functional.interpolate(x, scale_factor=self.scale_factor, mode='nearest') #resize the feature map using a nearest-neighbor upsampling

        return x


In [6]:
class GlobalAveragePooling(nn.Module):
    def __init__(self):
        super(GlobalAveragePooling, self).__init__()
        
        
    def forward(self, x):
        
        x = torch.mean(x, dim=(-2, -1))# perform global average pooling along spatial dimensions using the mean
        return x


Sequencer2Dmodel-> Is an architecture that resambles vision transformer except for the fact that Sequencer2D model is using BiLSTM2D (splitting image in horizzontal and vertical data) instead of self attention. It started with patch embedding layeer so the images will be divided in 8x8 patches, than ,after a normalization, there is the first sequence block of 4 Sequencer , the output of this block converge in PatchMerging, where the patches are resembled together ,then there is a second sequencer block made of 3 sequencer2d, followed by pointwise linear layer,then other sequencer block with 8 sequencers2D, then other point wise linear layer and  the final sequencerBlock with 3 sequencer2D layer and at the end there are linears+ global avarage pooling layers in order to make the output dimension as expected-

The parameters are the one suggested by the paper

CustomCifar2, this custom function process imace into the dataset in order to take only the first five classes, I've tried this apporch because each batch processing was too slowly.

CustomCifar2, this custom function process imace into the dataset in order to take only the first five classes, I've tried this apporch because each batch processing was too slowly.

In [7]:
import torch.nn.functional as F
class Sequencer2DModel(nn.Module):
    def __init__(self, num_classes, in_channels):
        super(Sequencer2DModel, self).__init__()

        self.num_classes = num_classes
        

       
        self.patch_embedding_1 = PatchEmbedding(32, 8, in_channels, 16)#  patch embedding with an 8x8 kernel size for each patch
        self.ln_1 = nn.LayerNorm(16)

       
        self.sequencer_block_1 =  nn.Sequential(
            Sequencer2D(16, 48, 16, 96),
            Sequencer2D(96, 96, 96, 192),
            Sequencer2D(192, 192, 192,384),
            Sequencer2D(384, 192, 384,384)
        )


       
        self.patch_merging=PatchMerging( 49152,128,2)#it still depend on batch size!!! it is sooo wrong

      
        self.sequencer_block_2 =  nn.Sequential(
            Sequencer2D(384, 192, 3,384),
            Sequencer2D(384, 192, 384,384),
            Sequencer2D(384, 192, 384,384)
        )

       
        self.pw_linear_1 = PWLinearLayer( 384,384)
        
        
        self.sequencer_block_3 =  nn.Sequential(
            Sequencer2D(384, 192, 384,384),
            Sequencer2D(384, 192, 384,384),
            Sequencer2D(384, 192, 384,384),
            Sequencer2D(384, 192, 384,384),
            Sequencer2D(384, 192, 384,384),
            Sequencer2D(384, 192, 384,384),
            Sequencer2D(384, 192, 384,384),
            Sequencer2D(384, 192, 384,384),
            
            
           
        )

        self.pw_linear_2 = PWLinearLayer(384, 384)

        
        self.sequencer_block_4 =  nn.Sequential(
            Sequencer2D(384, 192, 384,384),
            Sequencer2D(384, 192, 384,384),
            Sequencer2D(384, 192, 384,384)
        )
        self.pw_linear_3 = PWLinearLayer(384, 384)


        
        self.ln_2 = nn.LayerNorm(384)

        
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 384))

        
        self.fc = nn.Linear(384, num_classes)
        
        
        # Apply He initialization to linear layers
        for m in self.modules():
            if isinstance(m, nn.Linear):
                init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

    def forward(self, x):
       
        x = x.permute(0,3,1,2).clone()  
        #x = x.to(torch.float32).permute(0,3,1,2)  
        x = self.patch_embedding_1(x)
        x = self.ln_1(x)
        x = F.relu(x)
        
        x = self.sequencer_block_1(x)
       
        print(len(x))
        
        #x = self.patch_merging(x)
        
        
        # x = self.sequencer_block_2(x)
        
        
        x = self.pw_linear_1(x)
        # x = F.relu(x)
        # x = self.sequencer_block_3(x)
        
        
        # x = self.pw_linear_2(x)
        # x = F.relu(x)
    
        # x = self.sequencer_block_4(x)
       
        # x = self.pw_linear_3(x)
        # x = F.relu(x)
        
        x = self.ln_2(x)
        
        x = self.global_avg_pool(x)
        
        x = x.view(x.size(0), -1)
        
        x = self.fc(x)

        #x = x.squeeze(-1).squeeze(-1)
        #x=x[:,0]
        #x=x[:,0]

        return x



In [8]:
from torchvision import datasets
import numpy as np
from torch.utils.data import Dataset

class CustomCIFAR2(Dataset):
    def __init__(self, root, train=True, transform=None, target_transform=None, download=False):
        super(CustomCIFAR2, self).__init__()
        self.cifar10 = datasets.CIFAR10(root, train=train, transform=transform, target_transform=target_transform, download=download)
        
        
        self.keep_classes = [0, 1,2]  

      
        self.data, self.targets = self.filter_classes()

    def filter_classes(self):
        mask = np.isin(self.cifar10.targets, self.keep_classes)
        data = [self.cifar10.data[i] for i, include in enumerate(mask) if include]
        targets = [self.cifar10.targets[i] for i, include in enumerate(mask) if include]
        return data, targets

    def __getitem__(self, index):
        img, target = self.data[index], self.targets[index]
        return img, target

    def __len__(self):
        return len(self.data)


Load the model, with trnsform.Compose function that helps me with preprocessing and data augmentation:

In [9]:
import torch.optim as optim

import torchvision.transforms as transforms
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Sequencer2DModel(num_classes=3, in_channels=3)


model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


batch_size = 128

num_epochs = 10



transform = transforms.Compose([
    
    transforms.RandomHorizontalFlip(),   # Randomly flip the image horizontally
    transforms.RandomRotation(15),      # Randomly rotate the image by up to 15 degrees
    
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.4),# Adjust brightness, contrast, saturation, and hue
    transforms.RandomResizedCrop(16),
    transforms.RandomResizedCrop(4),# Randomly crop and resize the image to 224x224
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.RandomAffine(degrees=90, translate=(0.4, 0.1)),# Randomly translate the image
    transforms.ToTensor(),  # Convert the image to a tensor
    
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image

])


#train_dataset = torchvision.datasets.CIFAR10(root='./cifar-10-batches-py', train=True, transform=transform)
custom_dataset = CustomCIFAR2(root='./cifar-10-batches-py', train=True, transform=transform, download=True)
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)


Files already downloaded and verified


In this part the code aims to train the model: it iterates over  the number of epochs and then over all the batchs in the datasets takes the images and the labels and pass it to the model ,get the model outputs, and reshape it in order to make it fit well the loss function(CrossEntropyLoss function , 'cause it is a multiclassification problem), then performs backward step, and calls the optimizer (Adam starting with 0.01 learning rate) ,then it takes the prediction and compere it with the true labels,after that the accuracy is computed simply by divided the true presiction to the total  number of images processed.

In [10]:
import torch.nn.functional as F
all_accuracy=[]
all_loss=[]
# beginning of training
try:
    for epoch in range(num_epochs):
        total_correct = 0
        total_samples = 0
        running_loss = 0.0  # take track of the loss 
        
        model.train()

        for i, data in enumerate(data_loader, 0):
            if i==195:
                continue
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            
        
            optimizer.zero_grad()
            
            inputs=inputs.to(torch.float32)
            outputs = model(inputs)
            
            

        
            loss = criterion(outputs, labels)
            all_loss.append(loss)
            
        
            loss.backward()
            optimizer.step()

            # Print or log gradients
            # print("Gradients (max/min/mean):")
            # for name, param in model.named_parameters():
            #     if param.grad is not None:
            #         print(f"{name}: {param.grad.max().item():.6f} / {param.grad.min().item():.6f} / {param.grad.mean().item():.6f}")
            
            running_loss += loss.item()
            predicted_probabilities = outputs.argmax(dim=1)
            print(predicted_probabilities)
            correct = (predicted_probabilities == labels).sum().item()
            total_correct += correct
            total_samples += labels.size(0)

            batch_accuracy = (correct / labels.size(0)) * 100.0
            all_accuracy.append(batch_accuracy)
            
            
            
            print(f"Epoch [{epoch+1}/{num_epochs}] Batch [{i}/{len(data_loader)}] Loss: {loss.item():.4f} Accuracy: {batch_accuracy:.2f}%")
            if i % 200 == 199: 
                print(f"Epoch [{epoch+1}/{num_epochs}] Batch [{i}/{len(data_loader)}] Loss: {loss.item():.4f} Accuracy: {batch_accuracy:.2f}%")
                print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 200:.3f}")
                running_loss = 0.0
            torch.cuda.empty_cache()
        epoch_accuracy = (total_correct / total_samples) * 100.0
        torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                # Add any other relevant information
            }, 'checkpoint.pth')
            
        print(f"Epoch [{epoch+1}/{num_epochs}] Accuracy: {epoch_accuracy:.2f}%")

    print("Finished Training")
except Exception as e:
    print(f"Error: {e}")

128
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2])
Epoch [1/10] Batch [0/118] Loss: 4.5098 Accuracy: 28.91%
128
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
Epoch [1/10] Batch [1/118] Loss: 67.7880 Accuracy: 34.38%
128
tensor(

KeyboardInterrupt: 

code used to save the model

In [ ]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle

model_pkl_file = "sequencer_model384M.pkl"

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model, file)

Code for evaluation of the model over the dataset, here it is charged the test dataset as It was doing eiht the train dataset. Then the data are iterating and processed into the model to see if the prediciton match the actual labels, the code is way similar to the one used in the training phase except for the part of optimizer/backward .

In [ ]:

import matplotlib.pyplot as plt
all_loss=torch.tensor(all_loss)

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(all_loss.detach().numpy(), label='Loss')
plt.xlabel('Epoch')
plt.title('Loss Over Time')
plt.legend()
plt.show()
plt.subplot(1, 2, 2)
plt.plot(all_accuracy, label='Accuracy')
plt.xlabel('Epoch')
plt.title('Accuracy Over Time')
plt.legend()

plt.show()